In [ ]:
!pip install -q -U watermark

In [ ]:
!pip install -q -r requirements.txt

In [ ]:
import json
import numpy as np
import nltk
import shutil
import evaluate
import transformers
import datasets
from datasets import load_dataset
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Remove as pastas, se já existirem. Isso evita erros ao executar o jupyter a partir da segunda vez em diante
try:
    shutil.rmtree('logs_treino')
    shutil.rmtree('resultados_treino')
    shutil.rmtree('modelo_challange')
except:
    print('As pastas não existem ou já foram removidas!')

As pastas não existem ou já foram removidas!


#Carregando e preparando os dados de texto para treinamento

In [ ]:
# Carrega os dados
dataset = load_dataset('json', data_files='drive/MyDrive/FIAP/Tech Challenge/Fase3/trn.json')

# Carregando o tokenizador
tokenizador = T5Tokenizer.from_pretrained("google/flan-t5-base",legacy = False)

# Carregando o LLM pré-treinado
modelo = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")

data_collator = DataCollatorForSeq2Seq(tokenizer = tokenizador, model = modelo)

# Todos os inputs vão receber como prefixo: "answer the question" (responda a questão)
prefix = "answer the question: "


Generating train split: 0 examples [00:00, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
modelo

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=768, out_features=2048, bias=False)
              (wi_1): Linear(in_features=768, out_features=2048, bias=False)
              (wo):

In [ ]:
import pandas as pd

df = pd.DataFrame(dataset['train'])

# Removendo valores nulos ou em branco na coluna 'content'
df_cleaned = df.dropna(subset=['content'])  # Remove valores NaN (nulos) na coluna 'content'

# Caso a coluna 'content' contenha strings vazias, use o seguinte:
df_cleaned = df_cleaned[df_cleaned['content'].str.strip() != '']

In [ ]:
df_cleaned.head(5)

,uid,title,content,target_ind,target_rel
0,0000031909,Girls Ballet Tutu Neon Pink,High quality 3 layer ballet tutu. 12 inches in...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
3,0001360000,Mog's Kittens,Judith Kerr&#8217;s best&#8211;selling adventu...,"[146, 147, 148, 149, 495]","[1.0, 1.0, 1.0, 1.0, 1.0]"
7,0000031895,Girls Ballet Tutu Neon Blue,Dance tutu for girls ages 2-8 years. Perfect f...,"[12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 2...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
12,000100039X,The Prophet,"In a distant, timeless place, a mysterious pro...","[329, 330, 331, 332, 333, 334, 335, 336, 337, ...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ..."
13,0001473905,Rightly Dividing the Word,--This text refers to thePaperbackedition.,"[181, 182, 307, 380, 381]","[1.0, 1.0, 1.0, 1.0, 1.0]"


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['uid', 'title', 'content', 'target_ind', 'target_rel'],
        num_rows: 2248619
    })
})

#Dividindo as amostras de treino e teste em partes menores,apenas para acelerar o tempo de treinamento do modelo

In [ ]:
df_amazon = df_cleaned.head(5000)

In [ ]:
df_amazon['content'].head(5)

,content
0,High quality 3 layer ballet tutu. 12 inches in...
3,Judith Kerr&#8217;s best&#8211;selling adventu...
7,Dance tutu for girls ages 2-8 years. Perfect f...
12,"In a distant, timeless place, a mysterious pro..."
13,--This text refers to thePaperbackedition.


In [ ]:
from datasets import Dataset, DatasetDict
# Converter o DataFrame para um Dataset do Hugging Face
hf_dataset = Dataset.from_pandas(df_amazon)

# Criar o DatasetDict
dataset_dict = DatasetDict({
    'train': hf_dataset
})

In [ ]:
dataset = dataset_dict["train"].train_test_split(test_size = 0.2)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['uid', 'title', 'content', 'target_ind', 'target_rel', '__index_level_0__'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['uid', 'title', 'content', 'target_ind', 'target_rel', '__index_level_0__'],
        num_rows: 1000
    })
})

In [ ]:
#Função de pré processamento
def pre_processamento(data):

    # Concatena o prefixo a cada pergunta na lista de perguntas fornecida em data["title"]
    inputs = [prefix + doc for doc in data["title"]]

    # Usa o tokenizer para converter as perguntas processadas em tokens com um comprimento máximo de 1000,
    # truncando as que forem mais longas
    model_inputs = tokenizador(inputs, max_length = 256, truncation = True)

    # Tokeniza as respostas fornecidas em data["content"] com um comprimento máximo de 5000,
    # truncando as que forem mais longas
    labels = tokenizador(text_target = data["content"], max_length = 2048, truncation = True)

    # Adiciona os tokens das respostas como rótulos no dicionário de entradas do modelo
    model_inputs["labels"] = labels["input_ids"]

    # Retorna o dicionário processado contendo tanto as entradas tokenizadas quanto os rótulos
    return model_inputs

# Aplica a função de pré-processamento ao dataset gerando o dataset tokenizado

In [ ]:
dataset_tokenized = dataset.map(pre_processamento, batched = True)

Map:   0%|          | 0/4000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [ ]:
dataset_tokenized

DatasetDict({
    train: Dataset({
        features: ['uid', 'title', 'content', 'target_ind', 'target_rel', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 4000
    })
    test: Dataset({
        features: ['uid', 'title', 'content', 'target_ind', 'target_rel', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
        num_rows: 1000
    })
})

In [ ]:
dataset_tokenized['train']['title'][15]


"Amazing Grace: The Story of America's Most Beloved Song"

In [ ]:
dataset_tokenized['train']['content'][15]

'This carefully crafted and finely probed book will stand as the definitive look at what is perhaps the most popular hymn in American history a song that Turner argues has "more than eleven hundred currently available albums featuring versions." Turner\'s previous books on music and musicians (Trouble Man: The Life and Death of Marvin Gaye; Hungry for Heaven: Rock and Roll and the Search for Redemption) have dealt with the religious themes behind the historical facts, and his newest is no exception. Turner begins by detailing the life of the song\'s author, John Newton, an 18th-century slave trader whose miraculous survival during an 11-hour storm at sea in 1748 sparked a religious conversion that led to his becoming a minister (and later an avowed abolitionist) and to writing the hymn in 1773. Turner\'s examination of Newton\'s life and how it influenced the words of "Amazing Grace" gives an added resonance to the second half of his book. From the song\'s early 20th-century popularity

#Fazer uma pergunta teste para verificar a resposta do modelo antes do treinamento

In [ ]:
texto_entrada = "what the book called Amazing Grace: The Story of America's Most Beloved Song says?"


In [ ]:
tokenizador = T5Tokenizer.from_pretrained("google/flan-t5-base",legacy = False)

# Carregando o LLM pré-treinado
modelo = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Tokeniza o input
texto_entrada_tokenizado = tokenizador(texto_entrada, return_tensors = "pt").input_ids


In [ ]:
# Gera a saída (previsão do modelo)
texto_saida_tokenizado = modelo.generate(texto_entrada_tokenizado,
                                                   max_length = 2000,
                                                   temperature = 0.3,
                                                   do_sample = True)


In [ ]:
texto_saida = tokenizador.decode(texto_saida_tokenizado[0],
                                           skip_special_tokens = True)

#Resposta gerada para a pergunta antes do treinamento

In [ ]:
print("Pergunta:", texto_entrada)
print("Resposta:", texto_saida)

Pergunta: what the book called Amazing Grace: The Story of America's Most Beloved Song says?
Resposta: a symphony


In [ ]:
# O pacote "punkt" é específico para a tarefa de tokenização, que envolve a divisão de um texto
# em uma lista de sentenças
nltk.download("punkt", quiet = True)

True

In [ ]:
# Define a métrica
metric = evaluate.load("rouge")

# Função de cálculo das métricas

In [ ]:

def calcula_metricas(eval_preds):

    # Desempacota as predições e os rótulos do argumento eval_preds
    previsoes, labels = eval_preds

    # Substitui todos os valores diferentes de -100 em labels pelo ID do token de preenchimento
    labels = np.where(labels != -100,
                      labels,
                      tokenizador.pad_token_id)

    # Decodifica as previsões para texto, ignorando tokens especiais
    previsoes_decodificadas = tokenizador.batch_decode(previsoes,
                                                       skip_special_tokens = True)

    # Decodifica os labels para texto, ignorando tokens especiais
    labels_decodificados = tokenizador.batch_decode(labels,
                                                    skip_special_tokens = True)

    # Adiciona uma nova linha após cada sentença para as previsões decodificadas,
    # preparando-as para a avaliação ROUGE
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in previsoes_decodificadas]

    # Adiciona uma nova linha após cada label para as previsões decodificadas,
    # preparando-os para a avaliação ROUGE
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in labels_decodificados]

    # Calcula a métrica ROUGE entre as previções e os rótulos decodificados, utilizando um stemmer
    resultado = metric.compute(predictions = previsoes_decodificadas,
                               references = labels_decodificados,
                               use_stemmer = True)

    # Retorna o resultado da métrica ROUGE
    return resultado

In [ ]:

training_args = Seq2SeqTrainingArguments(output_dir = "resultados_treino",
                                             evaluation_strategy = "epoch",
                                             learning_rate = 3e-4,
                                             logging_dir = "logs_treino",
                                             logging_steps = 1,
                                             per_device_train_batch_size = 4,
                                             per_device_eval_batch_size = 2,
                                             weight_decay = 0.01,
                                             save_total_limit = 3,
                                             num_train_epochs = 3,
                                             predict_with_generate = True,
                                             push_to_hub = False)

# Define argumentos de treino

In [ ]:
# Define o treinador
trainer = Seq2SeqTrainer(model = modelo,
                             args = training_args,
                             train_dataset = dataset_tokenized["train"],
                             eval_dataset = dataset_tokenized["test"],
                             tokenizer = tokenizador,
                             data_collator = data_collator,
                             compute_metrics = calcula_metricas)

#Treinamento do modelo

In [ ]:
%%time
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,3.146200,2.940970,0.070006,0.014162,0.058029,0.058011
2,2.952800,2.878586,0.057666,0.013956,0.049801,0.049588
3,2.739100,2.862238,0.060774,0.015190,0.052444,0.052274


CPU times: user 27min 5s, sys: 1min 14s, total: 28min 19s
Wall time: 28min 48s


TrainOutput(global_step=3000, training_loss=3.0805524774193764, metrics={'train_runtime': 1726.0038, 'train_samples_per_second': 6.952, 'train_steps_per_second': 1.738, 'total_flos': 421398604763136.0, 'train_loss': 3.0805524774193764, 'epoch': 3.0})

#Deploy e uso do modelo

In [ ]:
# Salvando o modelo
trainer.save_model('modelo_challange')

In [ ]:
# Carrega o tokenizador final salvo em disco
tokenizador_final = AutoTokenizer.from_pretrained('modelo_challange')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
# Carrega o modelo final salvo em disco
modelo_final = AutoModelForSeq2SeqLM.from_pretrained('modelo_challange')

In [ ]:
modelo_final

In [ ]:
# Texto de entrada
texto_entrada = "what the book called Amazing Grace: The Story of America's Most Beloved Song says?"

In [ ]:
# Tokeniza o input
texto_entrada_tokenizado = tokenizador_final(texto_entrada, return_tensors = "pt").input_ids

In [ ]:
# Gera a saída (previsão do modelo)
texto_saida_tokenizado = modelo_final.generate(texto_entrada_tokenizado,
                                                   max_length = 2000,
                                                   temperature = 0.3,
                                                   do_sample = True)

In [ ]:
texto_saida_tokenizado

tensor([[   0,    3,  184,  ...,  184, 4663, 4613]])

In [ ]:
# Decode da saída
texto_saida = tokenizador_final.decode(texto_saida_tokenizado[0],
                                           skip_special_tokens = True)

In [ ]:
# Resultado
print("Pergunta:", texto_entrada)
print("Resposta:", texto_saida)

Pergunta: what the book called Amazing Grace: The Story of America's Most Beloved Song says?
Resposta: &#8220;Amazing Graceis a book that will be a welcome addition to any collection of Christian music collections.&#8221; (New York Times Book Review)&#8220;Amazing Graceis a book that will be a welcome addition to any collection of Christian music collections.&#8221; (New York Times Book Review)&#8220;Amazing Graceis a book that will be a welcome addition to any collection of Christian music collections.&#8221; (New York Times Book Review)&#8220;Amazing Graceis a book that will be a welcome addition to any collection of Christian music collections.&#8221; (New York Times Book Review)&#8220;Amazing Graceis a book that will be a welcome addition to any collection of Christian music collections.&#8221; (San Francisco Chronicle)&#8220;Amazing Graceis a book that will be a welcome addition to any collection of Christian music collections.&#8221; (San Francisco Chronicle)&#8220;Amazing Gracei